# Colab Setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
"""
Change directory to where this file is located
"""
#%cd '/content/drive/MyDrive/DLSP/Drna-master'

'\nChange directory to where this file is located\n'

# Loading data to colab (zip)

In [ ]:
zip_path = "/content/drive/MyDrive/DLSP/Drna-master/data/TestData.zip"
!cp "{zip_path}" .
!unzip -q TestData.zip
!rm TestData.zip

zip_path = "/content/drive/MyDrive/DLSP/Drna-master/data/TrainData.zip"
!cp "{zip_path}" .
!unzip -q TrainData.zip
!rm TrainData.zip

warning [TrainData.zip]:  76 extra bytes at beginning or within zipfile
  (attempting to process anyway)
error [TrainData.zip]:  reported length of central directory is
  -76 bytes too long (Atari STZip zipfile?  J.H.Holm ZIPSPLIT 1.1
  zipfile?).  Compensating...
error:  expected central file header signature not found (file #116969).
  (please check that you have transferred or created the zipfile in the
  appropriate BINARY mode and that you have compiled UnZip properly)


# Dataloader Ny

In [ ]:
import torch.utils.data as data
import torchvision
from torchvision import transforms
from PIL import Image
from tqdm import tqdm
from time import sleep

class TransformData(data.Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform
        
    def __getitem__(self, index):
        x, y = self.data[index]
        if self.transform:
            x = self.transform(x)
        return x, y
    
    def __len__(self):
        return len(self.data)

#testdata = torchvision.datasets.ImageFolder("TestData", transform = test_transform)
#len(testdata)

## Train RESNET50 fine-tuning all parameters (Small Images) LR as paper

In [ ]:
import os
import torch.utils.data
import torch.nn.functional as F
import torchvision
from torchvision import transforms
from torch.nn import DataParallel
from datetime import datetime
from torch.optim.lr_scheduler import MultiStepLR
import sys
sys.path.append("/content/drive/MyDrive/DLSP/Drna-master")
from config import BATCH_SIZE, PROPOSAL_NUM, SAVE_FREQ, LR, WD, resume, save_dir, INPUT_SIZE, INPUT_SIZE_s, VAL_PROCENT, RESIZED_SIZE, RESIZED_SIZE_s
from torch import nn
#from core import resnet as model
#from core import model_vgg as model
#from core import data_loader
#from core.utils import init_log, progress_bar
from PIL import Image
import numpy as np

BATCH_SIZE = 20
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
start_epoch = 1
save_dir = "/content/drive/MyDrive/DLSP/models/"
save_dir = os.path.join(save_dir, datetime.now().strftime('%Y%m%d_%H%M%S'))
if os.path.exists(save_dir):
    raise NameError('model dir exists!')
os.makedirs(save_dir)
print(save_dir)
#logging = init_log(save_dir)
#_print = logging.info

################################################ DATA #############################

train_transform = transforms.Compose([
    transforms.Resize(RESIZED_SIZE_s, Image.BILINEAR),
    transforms.RandomCrop(INPUT_SIZE_s),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.485, 0.456, 0.406),
                                 std=(0.229, 0.224, 0.225))
    ])


test_transform = transforms.Compose([
    transforms.Resize(RESIZED_SIZE_s, Image.BILINEAR),
    transforms.CenterCrop(INPUT_SIZE_s),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.485, 0.456, 0.406),
                                 std=(0.229, 0.224, 0.225))
    ])




Full_train_dataset = torchvision.datasets.ImageFolder("TrainData", transform = None)
train_nr = len(Full_train_dataset)
train_dataset, val_dataset = torch.utils.data.random_split(Full_train_dataset,
                                               [train_nr - int(np.ceil(train_nr*(VAL_PROCENT))), int(np.ceil(train_nr*(VAL_PROCENT)))],
                                               generator=torch.Generator().manual_seed(12))

train_dataset =  TransformData(train_dataset, transform = train_transform)
val_dataset =  TransformData(val_dataset, transform = test_transform)


test_dataset = torchvision.datasets.ImageFolder("TestData", transform = test_transform)


trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE,
                                          shuffle=True, num_workers=2, drop_last=False)

validationloader = torch.utils.data.DataLoader(val_dataset, batch_size=BATCH_SIZE,
                                          shuffle=False, num_workers=2, drop_last=False)


testloader = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE,
                                         shuffle=False, num_workers=2, drop_last=False)

print("Number of training data:", len(train_dataset))
print("Number of validation data:", len(val_dataset))
print("Number of test data:", len(test_dataset))

################################################ DATA #############################

# define model
net = torchvision.models.resnet50(pretrained=True)
#net = model.resnet152(pretrained=True)
# CHANING THE LAST FC-LAYER TO FIT 10 CLASSES
num_ftrs = net.fc.in_features
net.fc = nn.Linear(num_ftrs, 10)

resume = "/content/drive/MyDrive/DLSP/models/TRAIN1/TRAIN1/017.ckpt"
if resume:
    ckpt = torch.load(resume)
    net.load_state_dict(ckpt['net_state_dict'])
    start_epoch = ckpt['epoch'] + 1

criterion = torch.nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(net.parameters(), lr = 0.01)
net = net.cuda()
net = DataParallel(net)

for epoch in range(start_epoch, 30):
    # begin training
    print('--' * 50)
    net.train()
    train_loss = 0
    train_correct = 0
    total = 0
    with tqdm(trainloader, unit="batch") as tepoch:
        for data in tepoch:
            tepoch.set_description(f"Epoch {epoch}")

            img, label = data[0].cuda(), data[1].cuda()
            batch_size = img.size(0)
            optimizer.zero_grad()
            outputs = net(img)
            loss = criterion(outputs,label)
            loss.backward()
            optimizer.step()
            # calculate accuracy
            _, predict = torch.max(outputs, 1)
            total += batch_size
            train_correct += torch.sum(predict.data == label.data)
            train_loss += loss.item() * batch_size
            tepoch.set_postfix(loss=train_loss/total, accuracy=100. * float(train_correct)/total)
            sleep(0.1)
    # Printing status after each epoch and perform validation    
    if epoch % SAVE_FREQ == 0:
        train_acc = float(train_correct) / total
        train_loss = train_loss / total

        print(
            'epoch:{} - train loss: {:.3f} and train acc: {:.3f} total sample: {}'.format(
                epoch,
                train_loss,
                train_acc,
                total))
            
	      # evaluate on test set
        test_loss = 0
        test_correct = 0
        total = 0
        net.eval()
        for i, data in enumerate(validationloader):
            img, label = data[0].cuda(), data[1].cuda()
            batch_size = img.size(0)
            
            outputs = net(img)
            loss = criterion(outputs,label)
            
            
            # calculate accuracy
            _, predict = torch.max(outputs, 1)
            total += batch_size
            test_correct += torch.sum(predict.data == label.data)
            test_loss += loss.item() * batch_size

        test_acc = float(test_correct) / total
        test_loss = test_loss / total
        print(
            'epoch:{} - test loss: {:.3f} and test acc: {:.3f} total sample: {}'.format(
                epoch,
                test_loss,
                test_acc,
                total))
        

	      # save model
        net_state_dict = net.module.state_dict()
        if not os.path.exists(save_dir):
            os.mkdir(save_dir)
        torch.save({
            'epoch': epoch,
            'train_loss': train_loss,
            'train_acc': train_acc,
            'test_loss': test_loss,
            'test_acc': test_acc,
            'net_state_dict': net_state_dict},
            os.path.join(save_dir, '%03d.ckpt' % epoch))

print('finishing training')

/content/drive/MyDrive/DLSP/models/20211206_014156


/usr/local/lib/python3.7/dist-packages/torchvision/transforms/transforms.py:288: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  "Argument interpolation should be of type InterpolationMode instead of int. "


Number of training data: 93566
Number of validation data: 23392
Number of test data: 50182


Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

...

Epoch 20: 100%|██████████| 4679/4679 [45:08<00:00,  1.73batch/s, accuracy=80.3, loss=0.586]

epoch:20 - train loss: 0.586 and train acc: 0.803 total sample: 93566


epoch:20 - test loss: 1.722 and test acc: 0.576 total sample: 23392
----------------------------------------------------------------------------------------------------


Epoch 21:  41%|████      | 1930/4679 [18:35<26:28,  1.73batch/s, accuracy=82.1, loss=0.526]


KeyboardInterrupt: ignored

## Test-acc and loss

In [ ]:
import os
import torch.utils.data
import torch.nn.functional as F
import torchvision
from torchvision import transforms
from torch.nn import DataParallel
from datetime import datetime
from torch.optim.lr_scheduler import MultiStepLR
import sys
sys.path.append("/content/drive/MyDrive/DLSP/Drna-master")
from config import BATCH_SIZE, PROPOSAL_NUM, SAVE_FREQ, LR, WD, resume, save_dir, INPUT_SIZE, INPUT_SIZE_s, VAL_PROCENT, RESIZED_SIZE, RESIZED_SIZE_s
from torch import nn
#from core import resnet as model
#from core import model_vgg as model
#from core import data_loader
#from core.utils import init_log, progress_bar
from PIL import Image
import numpy as np

BATCH_SIZE = 20
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
start_epoch = 1
save_dir = "/content/drive/MyDrive/DLSP/models/"
save_dir = os.path.join(save_dir, datetime.now().strftime('%Y%m%d_%H%M%S'))
if os.path.exists(save_dir):
    raise NameError('model dir exists!')
os.makedirs(save_dir)
print(save_dir)
#logging = init_log(save_dir)
#_print = logging.info

################################################ DATA #############################


test_transform = transforms.Compose([
    transforms.Resize(RESIZED_SIZE_s, Image.BILINEAR),
    transforms.CenterCrop(INPUT_SIZE_s),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.485, 0.456, 0.406),
                                 std=(0.229, 0.224, 0.225))
    ])


test_dataset = torchvision.datasets.ImageFolder("TestData", transform = test_transform)


testloader = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE,
                                         shuffle=False, num_workers=2, drop_last=False)

print("Number of test data:", len(test_dataset))

################################################ DATA #############################

# define model
net = torchvision.models.resnet50(pretrained=True)
#net = model.resnet152(pretrained=True)
# CHANING THE LAST FC-LAYER TO FIT 10 CLASSES
num_ftrs = net.fc.in_features
net.fc = nn.Linear(num_ftrs, 10)

resume = "/content/drive/MyDrive/DLSP/models/BestModelSoFar/016.ckpt"
if resume:
    if torch.cuda.is_available():
      ckpt = torch.load(resume)
    else:
      ckpt = torch.load(resume,map_location=torch.device('cpu'))
    net.load_state_dict(ckpt['net_state_dict'])
    start_epoch = ckpt['epoch'] + 1

criterion = torch.nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(net.parameters(), lr = 0.01)
if torch.cuda.is_available():
  net = net.cuda()
net = DataParallel(net)

print('--' * 50)
net.eval()

test_loss = 0
test_correct = 0
total = 0
with tqdm(testloader, unit="batch") as tepoch:
    for data in tepoch:
        tepoch.set_description(f"Testing..")
        img, label = data[0], data[1]
        if torch.cuda.is_available():
            img, label = data[0].cuda(), data[1].cuda()
        batch_size = img.size(0)
        outputs = net(img)
        loss = criterion(outputs,label)
        # calculate accuracy
        _, predict = torch.max(outputs, 1)
        total += batch_size
        test_correct += torch.sum(predict.data == label.data)
        test_loss += loss.item() * batch_size
        tepoch.set_postfix(loss=test_loss/total, accuracy=100. * float(test_correct)/total)
        sleep(0.1)

test_acc = float(test_correct) / total
test_loss = test_loss / total

print('train loss: {:.3f} and train acc: {:.3f} total sample: {}'.format(
          test_loss,
          test_acc,
          total))

        

/content/drive/MyDrive/DLSP/models/20211203_112508


/usr/local/lib/python3.7/dist-packages/torchvision/transforms/transforms.py:288: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  "Argument interpolation should be of type InterpolationMode instead of int. "


Number of test data: 50182
----------------------------------------------------------------------------------------------------


Testing..: 100%|██████████| 2510/2510 [2:39:07<00:00,  3.80s/batch, accuracy=57, loss=1.56]


NameError: ignored